# 🤖 Cortex Complete Interactive Tutorial
## Learn by Doing: Snowflake LLM Text Generation

**Author:** Li Ma  
**Date:** February 24, 2026  
**Project:** DIA v2.0 - Direct Marketing Analytics Intelligence

---

## 📚 What You'll Learn

This interactive notebook teaches you how to:
1. ✅ Use Large Language Models (LLMs) in Snowflake
2. ✅ Generate creative marketing content
3. ✅ Summarize campaign data and reports
4. ✅ Analyze sentiment in customer feedback
5. ✅ Control creativity with temperature settings

## 🎯 Prerequisites

- Docker containers running (`docker-compose up`)
- Snowflake credentials configured in `.env` file
- Cortex Complete enabled in your Snowflake account

## 🧠 What is Cortex Complete?

**Cortex Complete** is Snowflake's LLM service that generates text using AI models like:
- **llama3-70b**: Best quality (70 billion parameters)
- **mistral-large**: Great reasoning
- **mistral-7b**: Fast and efficient

**Use Cases:**
- Write email subject lines
- Summarize data and reports
- Generate creative marketing content
- Analyze sentiment
- Answer questions (without accessing data)

---

**💡 Tip:** Run each cell with `Shift + Enter` and experiment with different prompts!

In [ ]:
# Install required packages for this notebook
# Run this cell once to install dependencies
import sys
import subprocess

packages = [
    'structlog',
    'python-dotenv',
    'snowflake-snowpark-python'
]

print("📦 Installing required packages...")
for package in packages:
    print(f"   Installing {package}...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
        print(f"   ✅ {package} installed")
    except subprocess.CalledProcessError as e:
        print(f"   ❌ Failed to install {package}: {e}")

print("\n✅ Installation complete!")
print("⚠️  If this is the first install, please RESTART THE KERNEL:")
print("   Jupyter menu: Kernel → Restart Kernel")

In [ ]:
# Setup Python paths and import libraries
import sys
import os

# Calculate the project paths dynamically
notebook_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebook_dir, '..'))
orchestrator_path = os.path.join(project_root, 'orchestrator')

# Add paths for both local and Docker environments
sys.path.insert(0, orchestrator_path)
sys.path.insert(0, project_root)
sys.path.insert(0, '/app')

print(f"📁 Python paths added:")
print(f"   Project Root: {project_root}")
print(f"   Orchestrator: {orchestrator_path}")

# Verify orchestrator path exists
if os.path.exists(orchestrator_path):
    print(f"   ✅ Orchestrator directory found")
else:
    print(f"   ⚠️  Orchestrator directory NOT found at: {orchestrator_path}")

# Core Python libraries
import json
from typing import Dict, List, Any, Optional
from dataclasses import dataclass

# Snowflake libraries
from snowflake.snowpark import Session

# Environment and logging
from dotenv import load_dotenv

# Try to import custom logger with fallback
try:
    from utils.logging import get_logger
    logger = get_logger(__name__)
    print(f"   ✅ Using custom structlog logger")
except ImportError as e:
    import logging
    logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logger = logging.getLogger(__name__)
    print(f"   ⚠️  Using standard logging (utils.logging not found)")

# Load environment variables from .env file
load_dotenv()

print("\n✅ All libraries imported successfully!")
print(f"   Python version: {sys.version.split()[0]}")

## 📦 Understanding the Response Data Model

The `CompleteResponse` dataclass holds all information from an LLM generation.

In [ ]:
@dataclass
class CompleteResponse:
    """
    Container for Cortex Complete (LLM) responses.
    
    Attributes:
        prompt (str): The input prompt/question
        completion (str): The generated text from LLM
        model (str): Which model was used (e.g., llama3-70b)
        metadata (Dict): Additional info (tokens, temperature, etc.)
        error (str): Error message if something went wrong
    """
    prompt: str
    completion: Optional[str] = None
    model: Optional[str] = None
    metadata: Optional[Dict[str, Any]] = None
    error: Optional[str] = None
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary (useful for JSON APIs)"""
        return {
            "prompt": self.prompt,
            "completion": self.completion,
            "model": self.model,
            "metadata": self.metadata,
            "error": self.error
        }

# Test it out!
sample_response = CompleteResponse(
    prompt="Write a catchy email subject line",
    completion="🎉 Limited Time: 50% Off Your Favorite Products!",
    model="llama3-70b",
    metadata={"temperature": 0.9, "tokens": 12}
)

print("✅ CompleteResponse created!")
print(f"   Prompt: {sample_response.prompt}")
print(f"   Generated: {sample_response.completion}")
print(f"   Model: {sample_response.model}")
print(f"   Temperature: {sample_response.metadata['temperature']}")

## 🔧 Import CortexComplete Service

Now let's import the `CortexComplete` class from the services module.

In [ ]:
# Import the CortexComplete service class
try:
    from services.cortex_complete import CortexComplete
    print("✅ CortexComplete class imported successfully!")
    print("   Ready to generate text with LLMs")
    print("\n📊 Available Models:")
    for model in CortexComplete.AVAILABLE_MODELS:
        print(f"   • {model}")
except ImportError as e:
    print(f"❌ Failed to import CortexComplete: {e}")
    print("\n💡 Troubleshooting:")
    print("   1. Make sure you ran Cell 2 (path setup)")
    print("   2. Check that orchestrator/services/cortex_complete.py exists")

## 🎨 Example 1: Generate Marketing Content

Let's use the LLM to generate creative marketing content. Higher temperature = more creative!

In [ ]:
# Generate a marketing tagline
with CortexComplete(model="llama3-70b", temperature=0.9) as llm:
    response = llm.complete(
        "Write a creative tagline for a data analytics platform that helps marketers understand email performance"
    )
    
    if response.error:
        print(f"❌ Error: {response.error}")
    else:
        print("✅ Generated Tagline:")
        print(f"\n   {response.completion}\n")
        print(f"📊 Metadata:")
        print(f"   Model: {response.model}")
        print(f"   Temperature: {response.metadata.get('temperature')}")
        print(f"   Prompt Length: {response.metadata.get('prompt_length')} chars")

## 📝 Example 2: Text Summarization

Summarize long text into concise summaries. Lower temperature = more focused!

In [ ]:
# Summarize campaign data
long_text = """
The Q1 2026 email marketing campaign ran from January 1 to March 31.
We sent 500,000 emails across 50 different customer segments.
Overall open rate was 24.5%, which exceeded our 20% target.
Click-through rate was 3.2%, slightly below our 3.5% target.
We generated $125,000 in revenue, representing a 15% increase from Q4 2025.
The top performing segments were Premium customers (35% open rate),
and Recent purchasers (28% open rate).
Mobile open rate was 18%, desktop was 32%.
Best send time was Tuesday 10 AM EST.
"""

with CortexComplete(model="mistral-large", temperature=0.3) as llm:
    response = llm.summarize(long_text, max_length=30)
    
    if response.error:
        print(f"❌ Error: {response.error}")
    else:
        print("✅ Summary Generated:")
        print(f"\n   {response.completion}\n")
        print(f"📊 Comparison:")
        print(f"   Original: {len(long_text.split())} words")
        print(f"   Summary: {len(response.completion.split())} words")
        print(f"   Reduction: {100 - (len(response.completion.split())/len(long_text.split())*100):.1f}%")

## ✉️ Example 3: Generate Email Subject Lines

Generate multiple creative subject lines for campaigns.

In [ ]:
# Generate email subject lines
with CortexComplete(model="llama3-70b", temperature=0.9) as llm:
    response = llm.generate_subject_lines(
        campaign_info="Spring collection launch, 25% off all items, free shipping over $50, limited 3-day sale",
        count=5
    )
    
    if response.error:
        print(f"❌ Error: {response.error}")
    else:
        print("✅ Generated Subject Lines:\n")
        print(response.completion)
        print(f"\n💡 Tip: Try different temperatures:")
        print("   • 0.9-1.0 = Very creative (best for marketing)")
        print("   • 0.5-0.7 = Balanced")
        print("   • 0.0-0.3 = Focused and consistent")

## 😊 Example 4: Sentiment Analysis

Analyze the sentiment of customer feedback or reviews.

In [ ]:
# Analyze sentiment of customer feedback
feedback_samples = [
    "This email platform is absolutely amazing! Best decision we made.",
    "The system crashed during our biggest campaign. Very disappointed.",
    "It works fine, nothing special. Does what it needs to do.",
]

with CortexComplete(model="mistral-7b", temperature=0.2) as llm:
    print("🔍 Sentiment Analysis Results:\n")
    
    for i, feedback in enumerate(feedback_samples, 1):
        response = llm.analyze_sentiment(feedback)
        
        print(f"{i}. Feedback: \"{feedback}\"")
        if response.error:
            print(f"   ❌ Error: {response.error}")
        else:
            print(f"   Result: {response.completion}")
        print()

## 🎯 Example 5: Compare Different Models

Let's see how different models respond to the same prompt.

In [ ]:
# Compare models side-by-side
prompt = "Explain what email click-through rate (CTR) means in simple terms."

models_to_test = ["llama3-70b", "mistral-large", "mistral-7b"]

print("📊 Comparing Model Responses:\n")
print("=" * 70)

for model_name in models_to_test:
    print(f"\n🤖 Model: {model_name}")
    print("-" * 70)
    
    try:
        with CortexComplete(model=model_name, temperature=0.5) as llm:
            response = llm.complete(prompt)
            
            if response.error:
                print(f"❌ Error: {response.error}")
            else:
                print(response.completion)
                print(f"\nLength: {len(response.completion.split())} words")
    except Exception as e:
        print(f"❌ Failed: {e}")

print("\n" + "=" * 70)
print("💡 Notice how different models have different styles and lengths!")

## 🌡️ Example 6: Temperature Experiment

See how temperature affects creativity. Same prompt, different temperatures!

In [ ]:
# Test different temperatures
prompt = "Write a short email subject line for a weekend flash sale"
temperatures = [0.0, 0.5, 1.0]

print("🌡️ Temperature Comparison:\n")
print("=" * 70)

with CortexComplete(model="llama3-70b") as llm:
    for temp in temperatures:
        print(f"\n🌡️  Temperature: {temp}")
        print("-" * 70)
        
        response = llm.complete(prompt, temperature=temp)
        
        if response.error:
            print(f"❌ Error: {response.error}")
        else:
            print(f"{response.completion}\n")

print("=" * 70)
print("\n💡 Observations:")
print("   • Temperature 0.0: Consistent, focused, predictable")
print("   • Temperature 0.5: Balanced creativity")
print("   • Temperature 1.0: Very creative, varied, unpredictable")

## 🎓 Summary: What You Learned

Congratulations! You've learned:

✅ **Cortex Complete Fundamentals**
- How to use LLMs in Snowflake
- Different models and their strengths
- Temperature control for creativity

✅ **Practical Applications**
- Generate marketing content
- Summarize long text
- Create email subject lines
- Analyze sentiment
- Compare model outputs

✅ **Python Programming**
- Context managers (`with` statement)
- Dataclasses for structured data
- Error handling patterns

---

## 🚀 Next Steps

**Try These Experiments:**
1. Generate subject lines for your own campaigns
2. Summarize your actual campaign reports
3. Test different models on the same task
4. Adjust temperature to see creativity variations

**Advanced Use Cases:**
1. **Content Generation Pipeline**
   - Generate → Review → Refine → Deploy
2. **Batch Processing**
   - Loop through multiple prompts
   - Save results to CSV/database
3. **A/B Testing**
   - Generate multiple variants
   - Test which performs better

---

## 🔗 Related Resources

- **Documentation:** `guides/02_STEP_2.1_CORTEX_SERVICES.md`
- **Service Code:** `orchestrator/services/cortex_complete.py`
- **Other Notebooks:**
  - `cortex_analyst_interactive.ipynb` - SQL generation
  - `cortex_search_interactive.ipynb` - Semantic search
  - `cortex_ml_interactive.ipynb` - Forecasting & anomalies

---

## 📝 Key Differences: Analyst vs Complete

**Cortex Analyst:**
- ❓ Question → 🔍 SQL → 📊 Real Data
- Use when: You need actual data from database

**Cortex Complete:**
- 💬 Prompt → 🤖 LLM → 📝 Generated Text
- Use when: You need creative content or summaries

**Use Together:**
```python
# Get data with Analyst
analyst = CortexAnalyst()
data = analyst.send_message("What was revenue last month?")

# Summarize with Complete
llm = CortexComplete()
summary = llm.summarize(str(data.results), max_length=50)
```

---

**Status:** ✅ Tutorial Complete  
**Next:** Try `cortex_search_interactive.ipynb` for semantic search!